In [8]:
import sys
sys.path.append("..")
from datasets import load_from_disk, Dataset
from transformers import AutoTokenizer,BertTokenizerFast
import numpy as np
from tqdm import tqdm, trange
import torch
import torch.nn.functional as F
from retrieval_model import BertEncoder,RobertaEncoder
from torch import nn
from torch.utils.data import (DataLoader,TensorDataset, SequentialSampler)
from tqdm import tqdm, trange
import pickle

In [9]:
p_encoder = BertEncoder.from_pretrained("/opt/ml/mrc-level2-nlp-08/retrieval/p_encoder")
q_encoder = BertEncoder.from_pretrained("/opt/ml/mrc-level2-nlp-08/retrieval/q_encoder")
#q_encoder_no = BertEncoder.from_pretrained("/opt/ml/mrc-level2-nlp-08/retrieval/q_encoder")
tokenizer =  AutoTokenizer.from_pretrained("kykim/bert-kor-base")
if torch.cuda.is_available():
        p_encoder.cuda()
        q_encoder.cuda()
        #q_encoder_no.cuda()

In [10]:
with open('/opt/ml/data/wiki_context_id_pair.bin','rb') as f:
    wiki_context_id = pickle.load(f)
with open('/opt/ml/data/wiki_id_context_pair.bin','rb') as f:
    wiki_id_context = pickle.load(f)
# with open('/opt/ml/data/dense_embedding_add_title.bin','rb') as f:
#     p_embs = pickle.load(f)
with open('/opt/ml/data/dense_embedding.bin','rb') as f:
    p_embs = pickle.load(f)
with open('/opt/ml/data/sentence_transformer_valid_60613.pickle','rb') as f:
    sen_tr = pickle.load(f)
wiki_corpus = list(wiki_context_id.keys())

In [11]:
if torch.cuda.is_available():
    p_embs_cuda = torch.Tensor(p_embs).to('cuda')


In [97]:
def get_relevant_doc_with_embs(q_encoder, p_embs, query, elastic_ids, elastic_scores,sen_tr):
    with torch.no_grad():
        q_encoder.eval()
        q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
        q_emb = q_encoder(**q_seqs_val).to('cuda')  #(num_query, emb_dim)


    dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
    rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
    rank = rank.cpu().numpy().tolist()

    es_id_score = {k:v for k,v in zip(elastic_ids,elastic_scores)}

    elastic_top_n = len(set(elastic_ids)) # 중복된 ids가 있다.
    hybrid_id_score = dict()
    
    sen_tr_id_score = dict()

    for i in range(len(sen_tr)):
        sen_tr_id_score[sen_tr[i]['corpus_id']] = sen_tr[i]['score']

    for i in range(len(rank)):
        dense_id = wiki_context_id[wiki_corpus[i]]

        if dense_id in es_id_score:
            lin_score = 1.1*dot_prod_scores[0][i].item() + es_id_score[dense_id]
            hybrid_id_score[dense_id] =  sen_tr_id_score[dense_id]*10 + lin_score
    
    return hybrid_id_score


In [98]:
with open('/opt/ml/data/elastic_valid_500.bin','rb') as f:
    elastic_valid = pickle.load(f)
with open('/opt/ml/data/elastic_valid_score_500.bin','rb') as f:
    elastic_valid_score = pickle.load(f)
querys = list(elastic_valid.keys())
hybrid = {}
# range(len(querys))
# querys[123]
for i in tqdm(range(len(querys))):
    q = querys[i]
    context = []
    #es,ds = get_relevant_doc_with_embs(q_encoder,p_embs_cuda,q,elastic_valid[q],elastic_valid_score)
    hybrid_id_score = get_relevant_doc_with_embs(q_encoder,p_embs_cuda,q,elastic_valid[q],elastic_valid_score[q],sen_tr[q])
    #hybrid_id_score = get_relevant_doc_with_embs(q_encoder,q_encoder_no,p_embs_cuda,p_embs_no_cuda,q,elastic_valid[q],elastic_valid_score[q])
    hybrid_id_score = list(hybrid_id_score.items())
    hybrid_id_score.sort(key = lambda x:x[1],reverse=True)
    hybrid_ids = list(map(lambda x: x[0], hybrid_id_score))
    hybrid[q] = hybrid_ids

    

100%|██████████| 240/240 [00:15<00:00, 15.55it/s]


In [99]:
with open("/opt/ml/data/hybrid_valid_retrieval_500.bin", "wb") as file:
    pickle.dump(hybrid,file)

In [57]:
len(hybrid[querys[0]])

500

In [30]:
with open('/opt/ml/data/elastic_train_1000.bin','rb') as f:
    elastic_valid = pickle.load(f)
with open('/opt/ml/data/elastic_train_score_1000.bin','rb') as f:
    elastic_valid_score = pickle.load(f)
querys = list(elastic_valid.keys())
hybrid = {}
# range(len(querys))
# querys[123]
for i in tqdm(range(len(querys))):
    q = querys[i]
    context = []
    #es,ds = get_relevant_doc_with_embs(q_encoder,p_embs_cuda,q,elastic_valid[q],elastic_valid_score)
    hybrid_id_score = get_relevant_doc_with_embs(q_encoder,p_embs_cuda,q,elastic_valid[q],elastic_valid_score[q])
    hybrid_id_score = list(hybrid_id_score.items())
    hybrid_id_score.sort(key = lambda x:x[1],reverse=True)
    hybrid_ids = list(map(lambda x: x[0], hybrid_id_score))
    hybrid[q] = hybrid_ids
    

100%|██████████| 3952/3952 [03:58<00:00, 16.58it/s]


In [31]:
with open("/opt/ml/data/hybrid_train_retrieval_1000.bin", "wb") as file:
    pickle.dump(hybrid,file)

In [101]:
with open('/opt/ml/data/sentence_transformer_test_60613.pickle','rb') as f:
    sen_tr = pickle.load(f)
with open('/opt/ml/data/elastic_test_500.bin','rb') as f:
    elastic_valid = pickle.load(f)
with open('/opt/ml/data/elastic_test_score_500.bin','rb') as f:
    elastic_valid_score = pickle.load(f)

querys = list(elastic_valid.keys())

hybrid = {}
# range(len(querys))
# querys[123]
for i in tqdm(range(len(querys))):
    q = querys[i]
    context = []
    #es,ds = get_relevant_doc_with_embs(q_encoder,p_embs_cuda,q,elastic_valid[q],elastic_valid_score)
    hybrid_id_score = get_relevant_doc_with_embs(q_encoder,p_embs_cuda,q,elastic_valid[q],elastic_valid_score[q],sen_tr[q])
    hybrid_id_score = list(hybrid_id_score.items())
    hybrid_id_score.sort(key = lambda x:x[1],reverse=True)
    hybrid_ids = list(map(lambda x: x[0], hybrid_id_score))
    hybrid[q] = hybrid_ids

100%|██████████| 600/600 [00:39<00:00, 15.25it/s]


In [102]:
with open("/opt/ml/data/hybrid_retrieval_st.bin", "wb") as file:
    pickle.dump(hybrid,file)

In [ ]:
def get_relevant_doc_with_embs(q_encoder, p_embs, query, elastic_ids, elastic_scores):
    with torch.no_grad():
        q_encoder.eval()
        q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
        q_emb = q_encoder(**q_seqs_val).to('cuda')  #(num_query, emb_dim)


    dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
    rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
    rank = rank.cpu().numpy().tolist()

    es_id_score = {k:v for k,v in zip(elastic_ids,elastic_scores)}

    elastic_top_n = len(set(elastic_ids)) # 중복된 ids가 있다.
    hybrid_id_score = dict()
    

    for i in range(len(rank)):
        dense_id = wiki_context_id[wiki_corpus[i]]
        #print(i, '-> ' ,dense_id)
        #if dense_id in es_id_score:
        if dense_id in es_id_score:

            hybrid_id_score[dense_id] =   dot_prod_scores[0][i].item() + es_id_score[dense_id]
    
    return hybrid_id_score


In [23]:
with open("/opt/ml/data/hybrid_retrieval_500.bin", "rb") as file:
    tt =  pickle.load(file)

In [24]:
with open("/opt/ml/data/hybrid_retrieval_300.bin", "rb") as file:
    v6 =  pickle.load(file)

In [25]:
q = list(tt.keys())

In [26]:
q[-2]

'제2캐나다기갑여단이 상륙한 곳은?'